#CheXpert

In [0]:
import os
import numpy as np
import time
import sys
import csv
import cv2
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as func
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torchvision.transforms import ToTensor,Resize
import torch.backends.cudnn as cudnn
import torchvision
import torch.optim as optim
from PIL import Image
from sklearn.metrics.ranking import roc_auc_score
import sklearn.metrics as metrics
import random

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

use_gpu = torch.cuda.is_available()

Mount google drive

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
baseFolder  = "drive/My Drive/CheXpert Dataset/CheXpert-v1.0-small/"

In [0]:
#Utility functions for cleaning the data

def cleanLabel(x):
    
    labelCount = 0    
    if x.Pleural_Effusion == 1:
        labelCount += 1
    if x.Edema == 1:
        labelCount += 1
    if x.Cardiomegaly ==1:
        labelCount += 1
    if x.Pneumonia == 1:
        labelCount += 1
    return labelCount
    
    

def getLabel(x):
    
    if x.Pleural_Effusion ==1:
        return "Pleural_Effusion"
    elif x.Edema == 1:
        return "Edema"
    elif x.Cardiomegaly==1:
        return "Cardiomegaly"
    elif x.Pneumonia == 1:
        return "Pneumonia"
    else:
        return "None"

In [0]:
cols = ['Path',
 'Sex',
 'Age',
 'View',
 'AP/PA',
 'No_Finding',
 'Enlarged_Cardiomediastinum',
 'Cardiomegaly',
 'Lung_Opacity',
 'Lung_Lesion',
 'Edema',
 'Consolidation',
 'Pneumonia',
 'Atelectasis',
 'Pneumothorax',
 'Pleural_Effusion',
 'Pleural_Other',
 'Fracture',
 'Support_Devices']
trainFile = pd.read_csv(os.path.join(baseFolder,'train.csv'), names = cols, header=0)
validFile = pd.read_csv(os.path.join(baseFolder,'valid.csv'), names = cols, header=0)

### Dataloader


In [0]:
labelMap = {"Pleural_Effusion":0, "Edema":1,"Cardiomegaly":2,"Pneumonia":3}

def getLabelDf(x):
    x = x[36:]          #To account for the extra "././" added before the Path variable
    x = df.loc[df.Path == x] 
    return labelMap[x.label.values[0]]

In [0]:
class LungDataset(Dataset):
    

    def __init__(self, csvFile, rootDir, transform = None):
        """
        Args:
            rootDir : Directory that has train, valid, train.csv and valid.csv
            csvFile : train.csv or valid.csv
        """
        
        self.rootDir = rootDir
        self.transform = transform
        
        cols = ['Path',
                 'Sex',
                 'Age',
                 'View',
                 'AP/PA',
                 'No_Finding',
                 'Enlarged_Cardiomediastinum',
                 'Cardiomegaly',
                 'Lung_Opacity',
                 'Lung_Lesion',
                 'Edema',
                 'Consolidation',
                 'Pneumonia',
                 'Atelectasis',
                 'Pneumothorax',
                 'Pleural_Effusion',
                 'Pleural_Other',
                 'Fracture',
                 'Support_Devices']
        
        self.df = pd.read_csv(os.path.join(rootDir,csvFile), names = cols, header=0)
        
        
        #Modifying the path variable
        self.df["Path"] = self.df.Path.apply(lambda x : x.replace('CheXpert-v1.0-small',"")[1:])
        
        #retaining important vars
        selectCols = ['Path',"View",'Sex',"Pleural_Effusion", "Edema","Cardiomegaly","Pneumonia"]
        self.df = self.df[selectCols]
        
        self.df['isClean'] = self.df.apply(lambda x : cleanLabel(x), axis = 1)
        #self.df["No"] = (self.df.isClean == 0).astype(int)
        #no_disease = self.df[self.df.isClean==0]
        #Retaining only samples with 1 disease
        self.df = self.df[self.df.isClean==1]
        #self.df = pd.concat([self.df,no_disease])
        
        #Creating the label variable
        self.df['label'] = self.df.apply(lambda x : labelMap[getLabel(x)], axis = 1)
                



    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        
        imgPath = os.path.join( self.rootDir, self.df.iloc[idx].Path)
        image = Image.open(imgPath).convert('RGB')
        
        label = self.df.iloc[idx].label

        if self.transform:
            image = self.transform(image)

        return image, label

In [0]:
trainDataset = LungDataset('valid.csv', baseFolder, transforms.Compose([Resize((256,256)), ToTensor()]))
validationDataset = LungDataset('valid.csv', baseFolder, transforms.Compose([Resize((256,256)), ToTensor()]))

In [0]:
e = next(iter(trainDataset))
e[0].size()

torch.Size([3, 256, 256])

In [0]:
train_data_loader = DataLoader(trainDataset, batch_size= 12, shuffle = True, num_workers = 4)
validation_loader = DataLoader(validationDataset, batch_size= 12, shuffle = True, num_workers = 4)

In [0]:
for image, label in train_data_loader:
    print(label)
    
    break;

tensor([1, 0, 2, 1, 2, 0, 0, 0, 1, 0, 0, 2])


## Logger

In [0]:
LOG_DIR = './logs'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

!if [ -f ngrok ] ; then echo "Ngrok already installed" ; else wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip > /dev/null 2>&1 && unzip ngrok-stable-linux-amd64.zip > /dev/null 2>&1 ; fi

In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

In [0]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print('Tensorboard Link: ' +str(json.load(sys.stdin)['tunnels'][0]['public_url']))"

Tensorboard Link: https://4f4d7fff.ngrok.io


In [0]:
# Code referenced from https://gist.github.com/gyglim/1f8dfb1b5c82627ae3efcfbbadb9f514
import tensorflow as tf
import numpy as np
import scipy.misc 
try:
    from StringIO import StringIO  # Python 2.7
except ImportError:
    from io import BytesIO         # Python 3.x


class Logger(object):
    
    def __init__(self, log_dir):
        """Create a summary writer logging to log_dir."""
        self.writer = tf.summary.FileWriter(log_dir)

    def scalar_summary(self, tag, value, step):
        """Log a scalar variable."""
        summary = tf.Summary(value=[tf.Summary.Value(tag=tag, simple_value=value)])
        self.writer.add_summary(summary, step)

    def image_summary(self, tag, images, step):
        """Log a list of images."""

        img_summaries = []
        for i, img in enumerate(images):
            # Write the image to a string
            try:
                s = StringIO()
            except:
                s = BytesIO()
            scipy.misc.toimage(img).save(s, format="png")

            # Create an Image object
            img_sum = tf.Summary.Image(encoded_image_string=s.getvalue(),
                                       height=img.shape[0],
                                       width=img.shape[1])
            # Create a Summary value
            img_summaries.append(tf.Summary.Value(tag='%s/%d' % (tag, i), image=img_sum))

        # Create and write Summary
        summary = tf.Summary(value=img_summaries)
        self.writer.add_summary(summary, step)
        
    def histo_summary(self, tag, values, step, bins=1000):
        """Log a histogram of the tensor of values."""

        # Create a histogram using numpy
        counts, bin_edges = np.histogram(values, bins=bins)

        # Fill the fields of the histogram proto
        hist = tf.HistogramProto()
        hist.min = float(np.min(values))
        hist.max = float(np.max(values))
        hist.num = int(np.prod(values.shape))
        hist.sum = float(np.sum(values))
        hist.sum_squares = float(np.sum(values**2))

        # Drop the start of the first bin
        bin_edges = bin_edges[1:]

        # Add bin edges and counts
        for edge in bin_edges:
            hist.bucket_limit.append(edge)
        for c in counts:
            hist.bucket.append(c)

        # Create and write Summary
        summary = tf.Summary(value=[tf.Summary.Value(tag=tag, histo=hist)])
        self.writer.add_summary(summary, step)
        self.writer.flush()
logger = Logger('./logs')


## Convolutional Neural Network

In [0]:
from torchvision import models
import torch.nn as nn
import torch.optim as optim

In [0]:
device = torch.device('cuda:0')

input_size = 196608
class Net(nn.Module):
    def __init__(self, num_classes):
        super(Net, self).__init__()

        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=4, stride = 2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride = 2),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride = 1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc1 = nn.Linear(4608, 256)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(256,num_classes)
    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0),-1)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        return out


In [0]:
def train(net, optimizer, criterion, train_loader, test_loader, epochs, size, model_name, plot):
    model = net.to(device)
    total_step = len(train_loader)
    overall_step = 0
    for epoch in range(epochs):
        correct = 0
        total = 0
        for i, (images, labels) in enumerate(train_loader):
            # Move tensors to configured device
            images = images.to(device)
            labels = labels.to(device)
            #Forward Pass
            outputs = model(images)
            
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            #print(loss)
            optimizer.step()
            _, prediction = torch.max(outputs,1)
            #print(outputs)
            #print(labels)
            #print(prediction)
            total += labels.size(0)
            correct += (prediction==labels).sum().item()
            accuracy = (labels == prediction.squeeze()).float().mean()
            overall_step+=1
            if (i+1) % 10 == 0:
              print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, epochs, i+1, total_step, loss.item()))
            if plot:
              info = { ('loss_' + model_name): loss.item(), ('accuracy_' + model_name): accuracy.item() }

              for tag, value in info.items():
                logger.scalar_summary(tag, value, overall_step+1)
        print('Training Accuracy of the network after {} epochs: {} %'.format(epoch, 100*correct/total))
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for i, (images, labels) in enumerate(test_loader):
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 289 test images: {} %'.format(100 * correct / total))


In [0]:
num_classes = 4
learning_rate = 0.0003
model = Net(4).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
epochs = 5


In [0]:
train(model, optimizer, criterion, train_data_loader, validation_loader, epochs, input_size , 'cnn', True)

Training Accuracy of the network after 0 epochs: 25.37313432835821 %
Training Accuracy of the network after 1 epochs: 41.791044776119406 %
Training Accuracy of the network after 2 epochs: 41.791044776119406 %
Training Accuracy of the network after 3 epochs: 43.28358208955224 %
Training Accuracy of the network after 4 epochs: 44.776119402985074 %
Accuracy of the network on the 289 test images: 43.28358208955224 %


In [0]:
model

Net(
  (layer1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(4, 4), stride=(2, 2))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=4608, out_features=256, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=256, out_features=4, bias=True)
  (softmax): Softmax()
)

In [0]:
from torch.autograd import Variable
def to_one_hot(y, n_dims=None):
    """ Take integer y (tensor or variable) with n dims and convert it to 1-hot representation with n+1 dims. """
    y_tensor = y.data if isinstance(y, Variable) else y
    y_tensor = y_tensor.type(torch.LongTensor).view(-1, 1)
    n_dims = n_dims if n_dims is not None else int(torch.max(y_tensor)) + 1
    y_one_hot = torch.zeros(y_tensor.size()[0], n_dims).scatter_(1, y_tensor, 1)
    y_one_hot = y_one_hot.view(*y.shape, -1)
    return Variable(y_one_hot) if isinstance(y, Variable) else y_one_hot